In [ ]:
from xtquant.xttrader import XtQuantTrader, XtQuantTraderCallback
from xtquant.xttype import StockAccount
import random

error_orders = []

class MyXtQuantTraderCallback(XtQuantTraderCallback):
    def on_disconnected(self):
        """
        连接状态回调
        :return:
        """
        print("connection lost")
    def on_stock_order(self, order):
        """
        委托信息推送
        :param order: XtOrder对象
        :return:
        """
        # 委托
        if order.order_status == 50:
            print(f"【已委托】 {order.order_type} 代码:{order.stock_code} 名称:{order.order_remark} 委托价格:{order.price:.2f} 委托数量:{order.order_volume} 订单编号:{order.order_id} 委托时间:{order.order_time}")
        elif order.order_status == 53 or order.order_status == 54:
            print(f"【已撤单】{order.order_type} 代码:{order.stock_code} 名称:{order.order_remark} 委托价格:{order.price:.2f} 委托数量:{order.order_volume} 订单编号:{order.order_id} 委托时间:{order.order_time}")

    def on_stock_trade(self, trade):
        """
        成交信息推送
        :param trade: XtTrade对象
        :return:
        """
        print(f"【已成交】{trade.order_type} 代码:{trade.stock_code} 名称:{trade.order_remark} 成交价格:{trade.traded_price:.2f} 成交数量:{trade.traded_volume} 成交编号:{trade.order_id} 成交时间:{trade.traded_time}")

    def on_order_error(self, data):
        if data.order_id in error_orders:
            return
        error_orders.append(data.order_id)
        print(f"【委托失败】错误信息:{data.error_msg.strip()}")

    def on_cancel_error(self, data):
        if data.order_id in error_orders:
            return
        error_orders.append(data.order_id)
        print(f"【撤单失败】错误信息:{data.error_msg.strip()}")


def create_trader(account_id, mini_qmt_path):
    # 创建session_id
    session_id = int(random.randint(100000, 999999))
    # 创建交易对象
    xt_trader = XtQuantTrader(mini_qmt_path, session_id)
    # 启动交易对象
    xt_trader.start()
    # 连接客户端
    connect_result = xt_trader.connect()

    if connect_result == 0:
        print(f"【miniQMT连接成功】路径:{mini_qmt_path}")

    # 创建账号对象
    account = StockAccount(account_id)
    # 订阅账号
    xt_trader.subscribe(account)
    print(f"【账号订阅成功】账号ID:{account_id}")
    # 注册回调类
    xt_trader.register_callback(MyXtQuantTraderCallback())

    return xt_trader, account

In [ ]:
xt_trader, account = create_trader("55003152", r"D:\国金QMT交易端模拟\userdata_mini")

In [ ]:
asset = xt_trader.query_stock_asset(account)
print('账号类型：', asset.account_type)
print('资金账号：', asset.account_id)
print('可用金额：', asset.cash)
print('冻结金额：', asset.frozen_cash)
print('持仓市值：', asset.market_value)
print('总资产：', asset.total_asset)

In [ ]:
positions = xt_trader.query_stock_positions(account)
for position in positions:
    print('账号类型:', position.account_type)
    print('资金账号:', position.account_id)
    print('证券代码:', position.stock_code)
    print('持仓数量:', position.volume)
    print('可用数量:', position.can_use_volume)
    print('开仓价:', position.open_price)
    print('市值:', position.market_value)
    print('冻结数量:', position.frozen_volume)
    print('在途股份:', position.on_road_volume)
    print('昨夜拥股:', position.yesterday_volume)
    print('成本价:', position.avg_price)
    print('------')


In [ ]:
orders = xt_trader.query_stock_orders(account, cancelable_only = False)
for order in orders:
  print('账号类型：', order.account_type)
  print('资金账号：', order.account_id)
  print('证券代码：', order.stock_code)
  print('订单编号：', order.order_id)
  print('柜台合同编号：', order.order_sysid)
  print('报单时间：', order.order_time)
  print('委托类型：', order.order_type)
  print('委托数量：', order.order_volume)
  print('报价类型：', order.price_type)
  print('委托价格：', order.price)
  print('成交数量：', order.traded_volume)
  print('成交均价：', order.traded_price)
  print('委托状态：', order.order_status)
  print('委托状态描述：', order.status_msg)
  print('-------')


In [ ]:
trades = xt_trader.query_stock_trades(account)
for trade in trades:
    print('账号类型:', trade.account_type)
    print('资金账号:', trade.account_id)
    print('证券代码:', trade.stock_code)
    print('委托类型:', trade.order_type)
    print('成交编号:', trade.traded_id)
    print('成交时间:', trade.traded_time)
    print('成交均价:', trade.traded_price)
    print('成交数量:', trade.traded_volume)
    print('成交金额:', trade.traded_amount)
    print('订单编号:', trade.order_id)
    print('柜台合同编号:', trade.order_sysid)
    print('-------')
